In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import math
%matplotlib inline

URL

In [208]:
urlConfirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
urlDeaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
urlCountries = 'https://restcountries.eu/rest/v2/all'
urlTemp = 'https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_3.23/crucy.1506241137.v3.23/countries/tmp/crucy.v3.23.1901.2014.{}.tmp.per'
wldMean = 795

Confirmed Cases

In [181]:
df_cc = pd.read_csv(urlConfirmed)
df_cc.head(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,969,981,989,998,1004,1029,1050,1076,1099,1122
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,762,762,762,762,763,763,763,763,764,764
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,58,60,61,69,70,70,71,74,81,84
5,NaN,Antigua and Barbuda,17.0608,-61.7964,0,0,0,0,0,0,...,25,25,25,25,25,25,25,25,25,25
6,NaN,Argentina,-38.4161,-63.6167,0,0,0,0,0,0,...,9931,10649,11353,12076,12628,13228,13933,14702,15419,16214
7,NaN,Armenia,40.0691,45.0382,0,0,0,0,0,0,...,5606,5928,6302,6661,7113,7402,7774,8216,8676,8927
8,Australian Capital Territory,Australia,-35.4735,149.0124,0,0,0,0,0,0,...,107,107,107,107,107,107,107,107,107,107
9,New South Wales,Australia,-33.8688,151.2093,0,0,0,0,3,4,...,3084,3086,3087,3090,3092,3089,3090,3092,3092,3095


In [182]:
indexesConfirmed = np.arange(2, len(df_cc.columns) - 1)
df_cc.drop(df_cc.columns[indexesConfirmed], axis=1, inplace=True)
df_cc.drop(df_cc.columns[0], axis=1, inplace=True)
df_cc.columns = ['Name','Cases']
dict_cc = {"Name": [], "Cases": []}

name = df_cc.iloc[0,0]
count = 0

for i in range(0, len(df_cc.index)-1):
    n = df_cc.iloc[i,0]
    if not n == name:
        dict_cc["Name"].append(name)
        dict_cc["Cases"].append(count)
        name = df_cc.iloc[i,0]
        count = 0
    count += df_cc.iloc[i, 1]

df_cc = pd.DataFrame.from_dict(dict_cc)
df_cc
        

,Name,Cases
0,Afghanistan,14525
1,Albania,1122
2,Algeria,9267
3,Andorra,764
4,Angola,84
...,...,...
187,South Sudan,994
188,Western Sahara,9
189,Sao Tome and Principe,479
190,Yemen,310


In [175]:
df_cc.head(10)

,Name,Cases
0,Afghanistan,14525
1,Albania,1122
2,Algeria,9267
3,Andorra,764
4,Angola,84
5,Antigua and Barbuda,25
6,Argentina,16214
7,Armenia,8927
8,Australia,107
9,Australia,3095


Deaths

In [186]:
df_de = pd.read_csv(urlDeaths)

In [193]:
df_de = pd.read_csv(urlDeaths)
indexesDeaths = np.arange(2, len(df_de.columns) - 1)
df_de.drop(df_de.columns[indexesDeaths], axis=1, inplace=True)
df_de.drop(df_de.columns[0], axis=1, inplace=True)
df_de.columns = ['Name', 'Deaths']

dict_de = {"Name": [], "Deaths": []}

name = df_de.iloc[0,0]
count = 0

for i in range(0, len(df_de.index)-1):
    n = df_de.iloc[i,0]
    if not n == name:
        dict_de["Name"].append(name)
        dict_de["Deaths"].append(count)
        name = df_de.iloc[i,0]
        count = 0
    count += df_de.iloc[i, 1]

df_de = pd.DataFrame.from_dict(dict_de)
df_de.drop(df_de.columns[0], axis=1, inplace=True)

In [194]:
df_de

,Deaths
0,249
1,33
2,646
3,51
4,4
...,...
187,10
188,1
189,12
190,77


Population

In [195]:
r = requests.get(urlCountries)
arrCountries = r.json()
names = []
populations = []
areas = []
dds = []
lats = []
longs = []
codes = []

for country in arrCountries:
    name = country["name"]
    population = country["population"]
    area = country["area"]
    latlng = country["latlng"]
    code = country["alpha3Code"]
    
    if name == None or population == None or area == None or latlng == None or code == None:
        continue
        
    names.append(name)
    populations.append(population)
    areas.append(area)
    dds.append((population/area))
    lats.append(latlng[0])
    longs.append(latlng[1])
    codes.append(code)
    
dictCountries = {"Name": names, "Code": codes, "Latitude":lats, "Longitude":longs, "Population": populations, "Area": areas, "Demographic_Density": dds}
df_co = pd.DataFrame.from_dict(dictCountries)

In [196]:
df_co

,Name,Code,Latitude,Longitude,Population,Area,Demographic_Density
0,Afghanistan,AFG,33.000000,65.0,27657145,652230.0,42.403976
1,Åland Islands,ALA,60.116667,19.9,28875,1580.0,18.275316
2,Albania,ALB,41.000000,20.0,2886026,28748.0,100.390497
3,Algeria,DZA,28.000000,3.0,40400000,2381741.0,16.962382
4,American Samoa,ASM,-14.333333,-170.0,57100,199.0,286.934673
...,...,...,...,...,...,...,...
235,Wallis and Futuna,WLF,-13.300000,-176.2,11750,142.0,82.746479
236,Western Sahara,ESH,24.500000,-13.0,510713,266000.0,1.919974
237,Yemen,YEM,15.000000,48.0,27478000,527968.0,52.044821
238,Zambia,ZMB,-15.000000,30.0,15933883,752612.0,21.171444


ICU BEDS

In [138]:
df_ib = pd.read_csv('DP_LIVE_31052020055440514.csv')

In [139]:
df_ib.drop(df_ib.columns[[1, 2, 3, 4, 7]], axis=1, inplace=True)
df_new = []

last = df_ib.iloc[0, 0]

for i in range(0, len(df_ib.index)):
    if df_ib.iloc[i, 0] != last:
        new = []
        new.append(df_ib.iloc[i - 1, 0])
        new.append(df_ib.iloc[i - 1, 2])
        
        df_new.append(new)
        last = df_ib.iloc[i, 0]
        
df_ib = pd.DataFrame(df_new, columns = ['Code', 'ICU_Beds'])

In [140]:
df_ib

,Code,ICU_Beds
0,AUS,3.84
1,AUT,7.37
2,BEL,5.64
3,CAN,2.50
4,CZE,6.63
5,DNK,2.50
6,FIN,3.28
7,FRA,5.98
8,DEU,8.00
9,GRC,4.21


Super Arquivo

In [197]:
df_super = pd.read_csv('SuperArquivo.csv')

In [142]:
df_super

,Country Name,Country Code,Series Name,Series Code,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],...,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019]
0,Argentina,ARG,Access to electricity (% of population),EG.ELC.ACCS.ZS,..,95.5110634674458,96.10791015625,96.384895324707,96.6386489868164,96.8673934936523,...,98.82,99.278694152832,99.4978561401367,99.4765548706055,100,99.8271179199219,99.9591217041016,100,100,..
1,Argentina,ARG,Adequacy of social insurance programs (% of to...,per_si_allsi.adq_pop_tot,..,..,..,..,..,..,...,39.6634007635173,32.5162579141245,35.7393513105137,42.8277408552611,..,..,..,..,..,..
2,Argentina,ARG,Adequacy of social safety net programs (% of t...,per_sa_allsa.adq_pop_tot,..,..,..,..,..,..,...,10.9621758748799,11.3343497372497,10.4701654975154,11.3135759745517,..,..,..,..,..,..
3,Argentina,ARG,Adjusted net national income per capita (curre...,NY.ADJ.NNTY.PC.CD,6606.31539239171,6159.20974612503,2047.20765727481,2748.63875996824,3248.68483504815,3850.90028844481,...,8806.20415998788,10954.8187178616,11152.2400601998,11154.9399965834,10453.3537408492,11843.9470848519,10892.4942331946,12344.0377978229,9631.7109854384,..
4,Argentina,ARG,"Air transport, passengers carried",IS.AIR.PSGR,8915627,5808770,5662905,6036360,6795484,6938436,...,9025035.31187357,8590839.84977289,9375669.73963746,11951276.8223926,12121912.818,14245183,15076354,16749271,18081937,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8451,Data from database: World Development Indicators,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_super = pd.read_csv('SuperArquivo.csv')
dict_super = {"Code": []}
code = df_super.iloc[0, 1]
dict_super["Code"].append(code)

for l in range(0, len(df_super.index)):
    series_name = df_super.iloc[l, 2]
    if not code == df_super.iloc[l, 1]:
        code = df_super.iloc[l, 1]
        
        if not str(code) == 'nan':
            dict_super["Code"].append(code)
        
    if not isinstance(series_name, str) or not isinstance(code, str):
        continue
    
    
    for c in range(len(df_super.columns)-1, -1, -1):
        value = df_super.iloc[l, c]
        if (isinstance(value, int) and math.isnan(value)):
            continue
        try:
            value = float(value)
        except:
            value = df_super.iloc[l, c]
            if not value == '..':
                if series_name in dict_super:
                    dict_super[series_name].append(np.nan)
                else:
                    dict_super[series_name] = [np.nan]
                break
            continue

        if series_name in dict_super:
            dict_super[series_name].append(value)
        else:
            dict_super[series_name] = [value]
            
        break

df_super = pd.DataFrame.from_dict(dict_super)

arr = ['Population ages 65 and above (% of total population)', 'Urban population (% of total population)', 'Demographic Density', 'Hospital beds (per 1,000 people)', 'Code']
df_super.drop(df_super.loc[:, ~df_super.columns.isin(arr)], axis=1, inplace=True)
df_super.drop([len(df_super.index)-1], inplace=True)
df_super = df_super.dropna()
df_super.set_index('Code', inplace=True)
df_super.loc['USA']

Hospital beds (per 1,000 people)                         2.900000
Population ages 65 and above (% of total population)    15.807654
Urban population (% of total population)                82.256000
Name: USA, dtype: float64

# MESCLANDO

In [257]:
#df_final["Cases_per_mil"] = []
r = requests.get('https://covid.ourworldindata.org/data/owid-covid-data.json')
dict_json_cpm = r.json()
dict_cpm = {"Code": [], "Cases_per_mil": []}
for key in dict_json_cpm.keys():
    cpm = dict_json_cpm[key][len(dict_json_cpm[key])-1]
    print(cpm)
    try:
        cpm = cpm['total_cases_per_million']
    except:
        cpm = cpm['total_tests']/(cpm['population']/10**6)
    dict_cpm["Code"].append(key)
    if cpm > wldMean:
        dict_cpm["Cases_per_mil"].append('Above')
    else:
        dict_cpm["Cases_per_mil"].append('Below')
df_cpm = pd.DataFrame.from_dict(dict_cpm)

{'location': 'Aruba', 'date': '2020-05-31', 'total_cases': 101, 'new_cases': 0, 'total_deaths': 3, 'new_deaths': 0, 'total_cases_per_million': 945.994, 'new_cases_per_million': 0.0, 'total_deaths_per_million': 28.099, 'new_deaths_per_million': 0.0, 'population': 106766.0, 'population_density': 584.8, 'median_age': 41.2, 'aged_65_older': 13.085, 'aged_70_older': 7.452, 'gdp_per_capita': 35973.781, 'diabetes_prevalence': 11.62}
{'location': 'Afghanistan', 'date': '2020-05-31', 'total_cases': 14525, 'new_cases': 866, 'total_deaths': 249, 'new_deaths': 3, 'total_cases_per_million': 373.121, 'new_cases_per_million': 22.246, 'total_deaths_per_million': 6.396, 'new_deaths_per_million': 0.077, 'population': 38928341.0, 'population_density': 54.422, 'median_age': 18.6, 'aged_65_older': 2.581, 'aged_70_older': 1.337, 'gdp_per_capita': 1803.987, 'cvd_death_rate': 597.029, 'diabetes_prevalence': 9.59, 'handwashing_facilities': 37.746, 'hospital_beds_per_100k': 0.5}
{'location': 'Angola', 'date': '

In [252]:
df_final = df_cc
df_final['Deaths'] = df_de['Deaths']
df_final['Cases_per_mil'] = df_cpm['Cases_per_mil']
df_final = df_final.join(df_co.set_index('Name'), on='Name')
df_final = df_final.join(df_super.set_index('Code'), on='Code')


In [254]:
df_final = df_final.dropna()

In [255]:
df_final

,Name,Cases,Deaths,Cases_per_mil,Code,Latitude,Longitude,Population,Area,Demographic_Density,...,Individuals using the Internet (% of population),"International tourism, number of arrivals","Life expectancy at birth, total (years)","Nurses and midwives (per 1,000 people)",Population ages 0-14 (% of total population),Population ages 65 and above (% of total population),Population density (people per sq. km of land area),Rural population (% of total population),"Tuberculosis case detection rate (%, all forms)",Urban population (% of total population)
2,Algeria,9267,646,Below,DZA,28.0,3.00,40400000.0,2381741.0,16.962382,...,49.038468,2657000.0,76.693000,1.5477,30.148679,6.362497,17.730075,27.371,80.0,72.629
4,Angola,84,4,Below,AGO,-12.5,18.50,25868000.0,1246700.0,20.749178,...,14.339079,218000.0,60.782000,0.4075,46.808925,2.216374,24.713052,34.486,61.0,65.514
6,Argentina,16214,528,Above,ARG,-34.0,-64.00,43590400.0,2780400.0,15.677744,...,74.294907,6942000.0,76.520000,2.5996,24.760934,11.117789,16.258510,8.130,87.0,91.870
7,Armenia,8927,127,Below,ARM,40.0,45.00,2994400.0,29743.0,100.675789,...,64.744884,1652000.0,74.945000,6.1070,20.633422,11.253818,103.680225,36.851,80.0,63.149
8,Australia,7192,103,Above,AUS,-27.0,133.00,24117360.0,7692024.0,3.135372,...,86.545049,9246000.0,82.748780,12.5508,19.190617,15.656475,3.247871,13.988,87.0,86.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,Sierra Leone,852,46,Below,SLE,8.5,-11.50,7075641.0,71740.0,98.628952,...,9.000000,57000.0,54.309000,0.2239,41.059639,2.966556,105.987171,57.945,75.0,42.055
183,Netherlands,6,0,Below,NLD,52.5,5.75,17019800.0,41850.0,406.685783,...,94.712074,18780000.0,81.760976,11.1839,16.108159,19.196193,511.475928,8.510,87.0,91.490
184,Malawi,279,4,Below,MWI,-13.5,34.00,16832910.0,118484.0,142.069056,...,13.782164,871000.0,63.798000,0.4386,43.902286,2.645435,192.440762,83.063,48.0,16.937
189,Sao Tome and Principe,479,12,Below,STP,1.0,7.00,187356.0,964.0,194.352697,...,29.931229,33400.0,70.170000,1.9242,42.519950,2.925678,219.820833,27.197,57.0,72.803


In [256]:
df_final.to_csv('final.csv')

In [11]:
df = pd.read_csv('final.csv')
df.set_index('Name', inplace=True)

In [24]:
df.loc['Brazil']

Unnamed: 0                                                       21
Cases                                                        498440
Deaths                                                        28834
Cases_per_mil                                                 Below
Code                                                            BRA
Latitude                                                        -10
Longitude                                                       -55
Population                                              2.06136e+08
Area                                                    8.51577e+06
Demographic_Density                                         24.2064
Hospital beds (per 1,000 people)                                2.2
Population ages 65 and above (% of total population)        8.92284
Urban population (% of total population)                     86.569
Name: Brazil, dtype: object